## How Neural Style Transfer (NST) Works
Neural Style Transfer (NST) is a technique that combines the content of one image with the artistic style of another using a pre-trained convolutional neural network (CNN). The key idea is to separate content and style features in deep layers of a CNN and then recombine them.

NST uses an optimization-based approach where:

- The content image defines the structure.
- The style image provides the artistic texture.
- A randomly initialized image (or a copy of the content image) is optimized to minimize the difference in content with the content image and the difference in style with the style image.


Let's start with the first step

#### Load a Pretrained CNN (VGG19)
- Use a pretrained VGG19 model.
- Extract specific layers to get content and style features.
- Make sure to freeze the weights so they don’t get updated during training.

🔹 Hints:
Store the feature maps from specific layers:
- conv4_2 for content features.
- conv1_1, conv2_1, conv3_1, conv4_1, conv5_1 for style features.

In [1]:
import torch
import torchvision

In [2]:
model = torchvision.models.vgg19(pretrained=True)

/opt/homebrew/Caskroom/miniforge/base/envs/fastai/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/fastai/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
class VGGFeatureExtractor():
    def __init__(model):
        # pass

    def forward(X):
        #pass